# **Two fold cross validation for logistic regression and baseline**

## Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.base import clone
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_score
import pandas as pd

## Load dataset

In [2]:
filename = 'heart_failure_clinical_records_dataset.csv'
df = pd.read_csv(filename)
y = df['DEATH_EVENT']
X = df.drop(['DEATH_EVENT'], axis = 1)
attributeNames = list(X.keys())
X = X.to_numpy()
y = y.to_numpy()

mu = np.mean(X, 0)
sigma = np.std(X, 0)
X = (X - mu) / sigma
X = (X - mu) / sigma
classNames = "DEATH_EVENT"
N, M = X.shape
C = 2

## Define our models

We are going to define two models:

- A baseline
- A logistic regression 
- A neural network

### Baseline

In [3]:
# In our dataset there are more entries of the class 0 (Survived)
# The best possible accuracy will be around 70% given our dataset
def baseline(dataset):
    dataset_index = dataset.index
    baseline = pd.Series(np.zeros(300)).astype(int)
    baseline = baseline.iloc[dataset_index]
    return baseline


### Logistic regression

In [4]:

log_reg = lambda regularization: LogisticRegression(penalty="l2", C= 1/regularization, max_iter= 5000)

In [5]:
def train_logistic_regression(model, X, y, X_test, y_test, X_par, regularization):
    test_error = np.empty(len(regularization))
    for s, lambda_value in enumerate(regularization):
        reg = model(lambda_value)
        reg.fit(X,y)
        y_pred = reg.predict(X_test)
        error = sum(y_pred != y_test) / len(y_test)
        test_error[s] = error
    
    return test_error


### ANN

In [6]:
nn = lambda n_hidden_units: torch.nn.Sequential(
                    torch.nn.Linear(M, n_hidden_units), #M features to H hiden units
                    torch.nn.Tanh(),   # 1st transfer function,
                    torch.nn.Linear(n_hidden_units, 1), # H hidden units to 1 output neuron
                    torch.nn.Sigmoid() # final tranfer function
                    )
loss_fn = torch.nn.BCELoss()

In [7]:
def train_neural_net(model, loss_fn, X, y, n_hidden_units,
                     n_replicates=1, max_iter = 10000, tolerance=1e-6):
    # Specify maximum number of iterations for training
    logging_frequency = 1000 # display the loss every 1000th iteration
    best_final_loss = 1e100
    for r in range(n_replicates):
        # Make a new net (calling model() makes a new initialization of weights) 
        net = model(n_hidden_units)
        
        # initialize weights based on limits that scale with number of in- and
        # outputs to the layer, increasing the chance that we converge to 
        # a good solution
        torch.nn.init.xavier_uniform_(net[0].weight)
        torch.nn.init.xavier_uniform_(net[2].weight)
                     
        # We can optimize the weights by means of stochastic gradient descent
        # The learning rate, lr, can be adjusted if training doesn't perform as
        # intended try reducing the lr. If the learning curve hasn't converged
        # (i.e. "flattend out"), you can try try increasing the maximum number of
        # iterations, but also potentially increasing the learning rate:
        #optimizer = torch.optim.SGD(net.parameters(), lr = 5e-3)
        
        # A more complicated optimizer is the Adam-algortihm, which is an extension
        # of SGD to adaptively change the learing rate, which is widely used:
        optimizer = torch.optim.Adam(net.parameters())
        
        # Train the network while displaying and storing the loss
        learning_curve = [] # setup storage for loss at each step
        old_loss = 1e6
        for i in range(max_iter):
            y_est = net(X) # forward pass, predict labels on training set
            loss = loss_fn(y_est, y) # determine loss
            loss_value = loss.data.numpy() #get numpy array instead of tensor
            learning_curve.append(loss_value) # record loss for later display
            
            # Convergence check, see if the percentual loss decrease is within
            # tolerance:
            p_delta_loss = np.abs(loss_value-old_loss)/old_loss
            if p_delta_loss < tolerance: break
            old_loss = loss_value
            
            # display loss with some frequency:
            # do backpropagation of loss and optimize weights 
            optimizer.zero_grad(); loss.backward(); optimizer.step()
            
            
        # display final loss
        
        if loss_value < best_final_loss: 
            best_net = net
            best_final_loss = loss_value
            best_learning_curve = learning_curve
            best_hidden_unit = n_hidden_units
        
    # Return the best curve along with its final loss and learing curve
    return best_net

## Two fold cross-validation

In [8]:

y.squeeze()

# Variables to control fold splits
K1 = 10
K2 = 10

# Initialize variables
lambda_interval = np.linspace(0.001, 0.1, 9) # Holds regularization values for our logistic regression model
hidden_units_interval = [1, 2, 3, 4, 5] # Holds hidden units range for our neural network
final_models_lambda = []
final_models_reg_error = []
final_models_hidden_unit = []
final_models_nn_error = []
final_baseline_error = []
Error_test = np.empty((K1,1))
Validation_error_nn = np.zeros((len(hidden_units_interval), K2))
Validation_error_reg = np.zeros((len(lambda_interval), K2)) 

Outer_fold = StratifiedKFold(n_splits = K1, shuffle = True)
Inner_fold =  StratifiedKFold(n_splits = K2, shuffle = True)

for i, (par_index, test_index) in enumerate(Outer_fold.split(X,y)):
    print('\nCrossvalidation outer fold: {0}/{1}'.format(i+1,K1))
    X_par, y_par = X[par_index,:], y[par_index]
    X_test, y_test = X[test_index,:], y[test_index]

    Generalization_error_reg = []
    Generalization_error_nn = []
    Generalization_error_base = []


    for j, (train_index, val_index) in enumerate(Inner_fold.split(X_par, y_par)):
        print('\nCrossvalidation inner fold: {0}/{1}'.format(j+1,K2))
        X_train, y_train = X_par[train_index, :], y_par[train_index]
        X_val, y_val = X_par[val_index], y_par[val_index]

        # Now we test our s models in regression
        for s, lambda_val in enumerate(lambda_interval):
            model  = log_reg(lambda_val)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_val)
            Validation_error_reg[s,j] = (len(y_val)/len(y_par))*(sum(y_pred != y_val)/len(y_pred))
        
        X_train_t = torch.tensor(X_train, dtype= torch.float)
        y_train_t = torch.tensor(y_train, dtype= torch.float)
        X_val_t = torch.tensor(X_val, dtype= torch.float)
        y_val_t = torch.tensor(y_val, dtype= torch.float)
        # Test for each hidden unit
        for h, n_hidden_units in enumerate(hidden_units_interval):
            net = train_neural_net(nn, loss_fn, X_train_t, y_train_t, n_hidden_units)
            y_sigmoid = net(X_val_t)
            y_test_est = (y_sigmoid>.5).type(dtype=torch.uint8)
            y_test_est = y_test_est.numpy()
            y_test_est = y_test_est.reshape(y_val.shape)
            # Determine errors and errors
            Validation_error_nn[h,j] = (len(y_val)/len(y_par)) * (sum(y_test_est != y_val)/len(y_val))
            
    
    # We compute for each model its generalization error
    for s in range(len(lambda_interval)):
        Generalization_error_reg.append(np.sum(Validation_error_reg[s, :]))

    for h in range(len(hidden_units_interval)):
        Generalization_error_nn.append(np.sum(Validation_error_nn[h, :]))

    # We select the best model and compute its test error with D_test
    min_index_reg = np.argmin(Generalization_error_reg)
    best_lambda = lambda_interval[min_index_reg]
    model = log_reg(best_lambda)
    model.fit(X_par, y_par)
    y_pred = model.predict(X_test)
    error_test_reg = sum(y_pred != y_test)/len(y_pred)

    X_par_t = torch.tensor(X_par, dtype= torch.float)
    y_par_t = torch.tensor(y_par, dtype= torch.float)
    X_test_t = torch.tensor(X_test, dtype= torch.float)
    y_test_t = torch.tensor(y_test, dtype= torch.float)

    min_index_nn = np.argmin(Generalization_error_nn)
    best_hidden_unit = hidden_units_interval[min_index_nn]
    net = train_neural_net(nn, loss_fn, X_par_t, y_par_t, best_hidden_unit)
    y_sigmoid = net(X_test_t)
    y_test_est = (y_sigmoid>.5).type(dtype=torch.uint8)
    y_test_est = y_test_est.numpy()
    y_test_est = y_test_est.reshape(y_test.shape)
    error_test_nn = sum(y_test_est != y_test)/len(y_test)

    y_pred_baseline = baseline(pd.Series(y_test))
    
    final_baseline_error.append(sum(y_pred_baseline != y_test)/len(y_pred_baseline))

    # We store the final models data
    final_models_lambda.append(best_lambda)
    final_models_reg_error.append(error_test_reg)
    final_models_hidden_unit.append(best_hidden_unit)
    final_models_nn_error.append(error_test_nn)

        



Crossvalidation outer fold: 1/3

Crossvalidation inner fold: 1/3


/home/erik/anaconda3/envs/ann/lib/python3.7/site-packages/torch/nn/modules/loss.py:512: UserWarning: Using a target size (torch.Size([132])) that is different to the input size (torch.Size([132, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)



Crossvalidation inner fold: 2/3


/home/erik/anaconda3/envs/ann/lib/python3.7/site-packages/torch/nn/modules/loss.py:512: UserWarning: Using a target size (torch.Size([133])) that is different to the input size (torch.Size([133, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)



Crossvalidation inner fold: 3/3


...

/home/erik/anaconda3/envs/ann/lib/python3.7/site-packages/torch/nn/modules/loss.py:512: UserWarning: Using a target size (torch.Size([133])) that is different to the input size (torch.Size([133, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)



Crossvalidation inner fold: 2/3


/home/erik/anaconda3/envs/ann/lib/python3.7/site-packages/torch/nn/modules/loss.py:512: UserWarning: Using a target size (torch.Size([133])) that is different to the input size (torch.Size([133, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)



Crossvalidation inner fold: 3/3


/home/erik/anaconda3/envs/ann/lib/python3.7/site-packages/torch/nn/modules/loss.py:512: UserWarning: Using a target size (torch.Size([134])) that is different to the input size (torch.Size([134, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
/home/erik/anaconda3/envs/ann/lib/python3.7/site-packages/torch/nn/modules/loss.py:512: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


In [11]:
data = {"Outer fold": range(1, K1 + 1),
        "h": final_models_hidden_unit,
        "E_test_nn": final_models_nn_error,
        "lambda": final_models_lambda,
        "E_test_reg": final_models_reg_error,
        "E_test_baseline": final_baseline_error}

df = pd.DataFrame(data)
df.set_index("Outer fold")

print(df)

range(1, 3)
   Outer fold  h  E_test_nn   lambda  E_test_reg  E_test_baseline
0           1  5   0.230000  0.00100     0.18000         0.320000
1           2  4   0.330000  0.00100     0.14000         0.320000
2           3  5   0.343434  0.02575     0.30303         0.323232
